## 1. Import and Load Packages 

In [1]:
# Load packages
import requests
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
import pandas as pd
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

### 2. Create exposure cohort in Big Query 

Set up connection

In [2]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'
cohort_table_id = 'exp_cohort'
cohort_id = 21

In [3]:
os.getcwd()

'/home/kbechler/notebooks'

In [4]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)

Create working dataset within the user's NERO project.

In [ ]:
client.create_dataset(f"{work_project_id}.{work_dataset_id}", exists_ok=True)

Helper function to visualize un-formatted notes in a slightly-better way

In [ ]:
def print_note(note_text):
    my_wrap = textwrap.TextWrapper(width = 80)
    [print(line) for line in my_wrap.wrap(text=note_text)]

### 3. Read file with SQL code from cohort derived from ATLAS. 

Cohort 1402 from ATLAS

In [ ]:
parametrized_sql = open('exp_cohort_dec21.sql','r').read()

In [ ]:
print(parametrized_sql)

Substitute each one of the parameters

In [ ]:
sql = re.sub(pattern=r'@temp_database_schema', 
             repl=f"`{work_project_id}.{work_dataset_id}`",
             string=parametrized_sql)

sql = re.sub(pattern=r'@target_database_schema', 
             repl=f"`{work_project_id}.{work_dataset_id}`",
             string=sql)

sql = re.sub(pattern=r'@target_cohort_table', 
             repl=cohort_table_id,
             string=sql)

sql = re.sub(pattern=r'@target_cohort_id', 
             repl=f"{cohort_id}",
             string=sql)

sql = re.sub(pattern=r'@vocabulary_database_schema', 
             repl=f"`{cdm_project_id}.{cdm_dataset_id}`",
             string=sql)

sql = re.sub(pattern=r'@cdm_database_schema', 
             repl=f"`{cdm_project_id}.{cdm_dataset_id}`",
             string=sql)

In [ ]:
sql = sql.lower()
sql = re.sub(pattern=r'create table', 
             repl="create or replace table",
             string=sql)

Cohort table MUST exist before executing this for the first time

In [ ]:
sql2 = """
CREATE OR REPLACE TABLE
`{work_project_id}.{work_dataset_id}.{cohort_table_id}`
(
cohort_definition_id INT64,
subject_id INT64,
cohort_start_date DATE,
cohort_end_date DATE
)

""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'cohort_table_id': cohort_table_id})

client.query(sql2).result();

### 4. Execute SQL

In [ ]:
%%time
client.query(sql).result()

### 5. Subset CDM using the cohort table 

Subset condition occurrence table to filter criteria based on notes.

In [ ]:
sql = """
    CREATE OR REPLACE TABLE
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
    SELECT
     c.concept_name as _{concept_id_prefix}_name,
     table_.*
    FROM
     `{cdm_project_id}.{cdm_dataset_id}.{table_id}` table_
    JOIN
     `{cdm_project_id}.{cdm_dataset_id}.concept` c
    ON
     c.concept_id = table_.{concept_id_prefix}_concept_id
    JOIN
     `{work_project_id}.{work_dataset_id}.{cohort_table_id}` cohort
    ON
     table_.person_id = cohort.subject_id
    WHERE
     cohort_definition_id = {cohort_id}
    """.format_map({'work_project_id': work_project_id,
                    'work_dataset_id': work_dataset_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'cohort_table_id': cohort_table_id,
                    'cohort_id': cohort_id,
                    'table_id': 'condition_occurrence',
                    'concept_id_prefix': 'condition'})
client.query(sql).result();

In [ ]:
sql = "select * from som-nero-nigam-starr.cdm_subset.condition_occurrence"
condition_df = client.query(sql).to_dataframe()

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_cohort"
cohort_df = client.query(sql).to_dataframe()

In [ ]:
print("Number of unique patients in cohort df: ", cohort_df.subject_id.nunique())
print("Number of unique patients in condition df: ",condition_df.person_id.nunique() )

In [ ]:
print(condition_df.condition_concept_id.nunique())

###### Condition codes: SLE
4055640: Lung disease with systemic lupus erythematosus  
4344158: Systemic lupus erythematosus with organ/system involvement  
4285717: SLE glomerulopehritis syndrome  
4063581: Drug-induced systemic lupus erythematosus  
4149913: Systemic lupus erythematosus with pericarditis  
257628: Systemic lupus erythematosus  

###### Condition codes: LN
4285717: SLE glomerulopehritis syndrome  

In [ ]:
# Condition concept id of SLE
# Mapped and descendants
sle_condition_list = [4055640, 4344158, 4285717 , 4063581, 4149913, 257628]

Confirms all patients have SLE condition concept id

In [ ]:
# Filter to patients who present with SLE
sle_condition = condition_df[condition_df['condition_concept_id'].isin(sle_condition_list)]
sle_condition.person_id.nunique()

In [ ]:
# Filter to patients who present with LN
ln_condition = condition_df[condition_df.condition_concept_id == 4285717]
ln_condition.person_id.nunique()

In [ ]:
sle_condition._condition_name.unique()

In [ ]:
ln_condition._condition_name.unique()

### 6. Calculate minimum date of SLE diagnosis (first instance of SLE condition id) 

In [ ]:
# Group by unique person and min condition concept ID for SLE
sle_min = sle_condition.groupby("person_id", as_index = False)['condition_start_DATETIME'].min()
print("Number of unique patients:", len(sle_min))
sle_min.head()

Merge with cohort and cohort start date to compare index date

In [ ]:
cohort_df = pd.merge(cohort_df, sle_min, left_on = 'subject_id', right_on = 'person_id', how = 'left')

In [ ]:
cohort_df.head()

In [ ]:
cohort_df.cohort_start_date = pd.to_datetime(cohort_df.cohort_start_date)

In [ ]:
# Calculate difference between cohort start date and condition start datetime
cohort_df['cohort_start_condition_start_dif'] = (cohort_df.cohort_start_date - cohort_df.condition_start_DATETIME).dt.days

In [ ]:
cohort_df.head()

In [ ]:
# Look at differences greater than -1
cohort_df[cohort_df.cohort_start_condition_start_dif < -1]

Need to look at this handful who have cohort_start_date prior to condition_start_DATETIME

### 7. Count how many of these patients have SLE in clinical notes

In [ ]:
# Pull note ids of these SLE patients
# Use som-rit-phi-starr-prod for note_ids
query = """
CREATE OR REPLACE TABLE
`{work_project_id}.{work_dataset_id}.sle_patient_notes` AS
SELECT
distinct(note.note_id) as note_id, 
person_id
FROM
`som-rit-phi-starr-prod.starr_omop_cdm5_deid_2021_12_13.note` note
WHERE
person_id IN
(SELECT 
subject_id
FROM
`{work_project_id}.{work_dataset_id}.exp_cohort`)
""".format_map({'cdm_project_id': cdm_project_id,
                'cdm_dataset_id': cdm_dataset_id,
                'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id})
client.query(query).result();

In [ ]:
# Pull note nlps of these
query = """
CREATE OR REPLACE TABLE
`{work_project_id}.{work_dataset_id}.sle_patient_note_nlp` AS
SELECT
*
FROM
`{cdm_project_id}.{cdm_dataset_id}.note_nlp` note_nlp
WHERE
note_id IN
(SELECT
note_id
FROM
`{work_project_id}.{work_dataset_id}.sle_patient_notes`)
AND
note_nlp.term_exists = 'Y' AND
note_nlp.term_modifiers = '[]' AND
note_nlp_concept_id IN
(SELECT
concept_id
FROM
`{work_project_id}.{work_dataset_id}.sle_concept_set`)
""".format_map({'cdm_project_id': cdm_project_id,
                'cdm_dataset_id': cdm_dataset_id,
                'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id})
client.query(query).result();

In [ ]:
# Create dataframe in Jupyter Notebook
query = """
SELECT
*
FROM
`{work_project_id}.{work_dataset_id}.sle_patient_note_nlp`
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id})

sle_patient_note_nlp = client.query(query).to_dataframe()
sle_patient_note_nlp.head()

In [ ]:
# Bring in person_id and note_id from patient notes

query = """
SELECT
* 
FROM 
`{work_project_id}.{work_dataset_id}.sle_patient_notes`
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id})

sle_patient_note = client.query(query).to_dataframe()
sle_patient_note.head()

In [ ]:
print("Number of unique patients:", sle_patient_note.person_id.nunique())

In [ ]:
# Merge with note nlp 
note_merge = pd.merge(sle_patient_note_nlp, sle_patient_note, on = 'note_id', how = 'left')

In [ ]:
print("Number of unique patients:", note_merge.person_id.nunique())

Create list of patients with cohort definition and mention of SLE condition concept id in clinical notes

In [ ]:
patients_complete = note_merge.person_id.unique()

In [ ]:
len(patients_complete)

In [ ]:
patients_complete_tuple = tuple(patients_complete)

### 8. Limit Cohort Table in ATLAS to those patients who have SLE condition present in clinical notes 

In [ ]:
# Limit cohort table
query = """
CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.exp_2022` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.exp_cohort` exp_cohort
WHERE
subject_id IN {}""".format(patients_complete_tuple)

client.query(query).result();

In [ ]:
# Create table of filtered patients
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_filtered_df = client.query(sql).to_dataframe()

In [ ]:
print("Number of patients:", len(exp_filtered_df))

### 9. Create outcome cohort from this exposure cohort 

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cohort_table_id = 'out_cohort'
cohort_id = 22
cdm_subset_dataset_id = 'cdm_subset'

In [ ]:
parametrized_sql = open('out_dec2021.sql','r').read()

In [ ]:
print(parametrized_sql)

In [ ]:
sql = re.sub(pattern=r'@temp_database_schema', 
             repl=f"`{work_project_id}.{work_dataset_id}`",
             string=parametrized_sql)

sql = re.sub(pattern=r'@target_database_schema', 
             repl=f"`{work_project_id}.{work_dataset_id}`",
             string=sql)

sql = re.sub(pattern=r'@target_cohort_table', 
             repl=cohort_table_id,
             string=sql)

sql = re.sub(pattern=r'@target_cohort_id', 
             repl=f"{cohort_id}",
             string=sql)

sql = re.sub(pattern=r'@vocabulary_database_schema', 
             repl=f"`{cdm_project_id}.{cdm_dataset_id}`",
             string=sql)

sql = re.sub(pattern=r'@cdm_database_schema', 
             repl=f"`{cdm_project_id}.{cdm_dataset_id}`",
             string=sql)

In [ ]:
sql = sql.lower()
sql = re.sub(pattern=r'create table', 
             repl="create or replace table",
             string=sql)

In [ ]:
sql2 = """
CREATE OR REPLACE TABLE
`{work_project_id}.{work_dataset_id}.{cohort_table_id}`
(
cohort_definition_id INT64,
subject_id INT64,
cohort_start_date DATE,
cohort_end_date DATE
)

""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'cohort_table_id': cohort_table_id})

client.query(sql2).result();

In [ ]:
%%time
client.query(sql).result()

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.out_cohort"
outcome_df = client.query(sql).to_dataframe()

In [ ]:
outcome_df.head()

In [ ]:
print("Number of unique patients:", len(outcome_df))

In [ ]:
# Merge with exp_filtered_df
outcome_merged = pd.merge(outcome_df, exp_filtered_df, on = 'subject_id', how = 'inner')

In [ ]:
outcome_merged.head()

In [ ]:
outcome_merged.columns = ['out_id', 'subject_id', 'out_cohort_start_date', 'out_cohort_end_date', 
                         'exp_id', 'exp_cohort_start_date', 'exp_cohort_end_date']

In [ ]:
outcome_merged.head()

Get list of patients from complete outcome cohort in ATLAS, then limited to the pre-filtered cohort based on 
clinical notes

In [ ]:
outcome_patients_tuple = tuple(outcome_merged.subject_id.unique())

In [ ]:
# Limit cohort table
query = """
CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.out_2022` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.out_cohort` out_cohort
WHERE
subject_id IN {}""".format(outcome_patients_tuple)

client.query(query).result();

In [ ]:
# Create table of filtered patients
sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
outcome_filtered_df = client.query(sql).to_dataframe()

In [ ]:
print("Number of patients in outcome cohort:", len(outcome_filtered_df))

### 10. Create CDM subset with correct exposure cohort filtered

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = 'starr_omop_cdm5_deid_20211213'
cdm_subset_dataset_id = 'cdm_subset'
cohort_table_id = 'exp_2022'
cohort_id = 21

In [ ]:
client.create_dataset(f"{work_project_id}.{cdm_subset_dataset_id}", exists_ok=True)

In [ ]:
copy_as_is = [
              #"attribute_definition", #empty
              "care_site",
              "cdm_source",
              "concept",
              "concept_ancestor",
              "concept_class",
              "concept_relationship",
              "concept_synonym",
              "cost", # empty
              "domain",
              "drug_strength",
              "metadata",
              "payer_plan_period", # empty
              "provider",
              "relationship",
              "source_to_concept_map",
              "specimen", # empty
              "vocabulary"]

In [ ]:
%%time
for table_id in copy_as_is:
    sql = """
        CREATE OR REPLACE TABLE 
         `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
        SELECT * 
        FROM 
         `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
        """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'work_project_id': work_project_id,
                        'cdm_subset_dataset_id': cdm_subset_dataset_id,
                        'table_id': table_id})
    
    job_config = bigquery.QueryJobConfig(dry_run=False, use_query_cache=True)
    query_job = client.query(sql, job_config)
    query_job.result()
    print("This query will process {} bytes.".format(query_job.total_bytes_processed))

First subset tables with an end date

In [ ]:
tables_with_end_date = ["condition_occurrence",
                        "device_exposure",
                        "drug_exposure",
                        "visit_detail",
                        "visit_occurrence"]

date_column_prefix = ["condition",
                      "device_exposure",
                      "drug_exposure",
                      "visit_detail",
                      "visit"]

concept_id_prefix = ["condition",
                     "device",
                     "drug",
                     "visit_detail",
                     "visit"]

In [ ]:
%%time
for i in range(len(tables_with_end_date)):
    
    table_id = tables_with_end_date[i]
    prefix_date = date_column_prefix[i]
    concept_id_prefix_table = concept_id_prefix[i]
    
    print(f'Subsetting {table_id}')
    
    sql = """
    CREATE OR REPLACE TABLE
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
    SELECT 
    c.concept_name as _{concept_id_prefix}_name,
    table_.*
    FROM
     `{cdm_project_id}.{cdm_dataset_id}.{table_id}` table_
    JOIN
     `{cdm_project_id}.{cdm_dataset_id}.concept` c
    ON
     c.concept_id = table_.{concept_id_prefix}_concept_id
    JOIN
     `{work_project_id}.kbechler_explore.{cohort_table_id}` cohort
    ON
     table_.person_id = cohort.subject_id
    WHERE
     cohort_definition_id = {cohort_id}
     AND table_.{prefix_date}_start_date <= cohort.cohort_end_date
    """.format_map({'work_project_id': work_project_id,
                    'work_dataset_id': work_dataset_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'cohort_table_id': cohort_table_id,
                    'cohort_id': cohort_id,
                    'table_id': table_id,
                    'prefix_date': prefix_date,
                    'concept_id_prefix': concept_id_prefix_table})
    
    client.query(sql).result();

In [ ]:
# Mirror and do tables without end date but acutally have end date of cohort end date

In [ ]:
tables_without_end_date = ["measurement",
                           #"note_nlp", 
                           "observation",
                           "procedure_occurrence"]

date_column_prefix = ["measurement",
                      #"note_nlp",
                      "observation",
                      "procedure"]

concept_id_prefix = ["measurement",
                     #"note_nlp",
                     "observation",
                     "procedure"]

In [6]:
%%time
for i in range(len(tables_without_end_date)):
    
    table_id = tables_without_end_date[i]
    prefix_date = date_column_prefix[i]
    concept_id_prefix_table = concept_id_prefix[i]
    
    print(f'Subsetting {table_id}')
    
    sql = """
    CREATE OR REPLACE TABLE
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
    SELECT
    c.concept_name as _{concept_id_prefix}_name,
    table_.*
    FROM
     `{cdm_project_id}.{cdm_dataset_id}.{table_id}` table_
    JOIN
     `{cdm_project_id}.{cdm_dataset_id}.concept` c
    ON
     c.concept_id = table_.{concept_id_prefix}_concept_id
    JOIN
     `{work_project_id}.{work_dataset_id}.{cohort_table_id}` cohort
    ON
     table_.person_id = cohort.subject_id
    WHERE
     cohort_definition_id = {cohort_id}
     AND table_.{prefix_date}_date 
     <= cohort.cohort_end_date
    """.format_map({'work_project_id': work_project_id,
                    'work_dataset_id': work_dataset_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'cohort_table_id': cohort_table_id,
                    'cohort_id': cohort_id,
                    'table_id': table_id,
                    'prefix_date': prefix_date,
                    'concept_id_prefix': concept_id_prefix_table})
    client.query(sql).result();

NameError: name 'tables_without_end_date' is not defined

Subset note_nlp

In [10]:
tables_without_end_date = ["note"]

date_column_prefix = ["note"]

concept_id_prefix = ["note_class"]

In [11]:
%%time
for i in range(len(tables_without_end_date)):
    
    table_id = tables_without_end_date[i]
    prefix_date = date_column_prefix[i]
    concept_id_prefix_table = concept_id_prefix[i]
    
    print(f'Subsetting {table_id}')
    
    sql = """
    CREATE OR REPLACE TABLE
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
    SELECT
    c.concept_name as _{concept_id_prefix}_name,
    table_.*
    FROM
     `som-rit-phi-starr-prod.starr_omop_cdm5_deid_2021_12_13.note` table_
    JOIN
     `{cdm_project_id}.{cdm_dataset_id}.concept` c
    ON
     c.concept_id = table_.{concept_id_prefix}_concept_id
    JOIN
     `{work_project_id}.{work_dataset_id}.{cohort_table_id}` cohort
    ON
     table_.person_id = cohort.subject_id
    WHERE
     cohort_definition_id = {cohort_id}
     AND table_.{prefix_date}_date 
     <= cohort.cohort_start_date
    """.format_map({'work_project_id': work_project_id,
                    'work_dataset_id': work_dataset_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'cohort_table_id': cohort_table_id,
                    'cohort_id': cohort_id,
                    'table_id': table_id,
                    'prefix_date': prefix_date,
                    'concept_id_prefix': concept_id_prefix_table})
    client.query(sql).result();

Subsetting note
CPU times: user 34.8 ms, sys: 1.3 ms, total: 36.1 ms
Wall time: 2min 2s


In [12]:
patient_notes = client.query(sql).to_dataframe()

Series([], dtype: object)

In [ ]:
len(patient_notes)

In [ ]:
note_patient_tuple = tuple(patient_notes.note_id)

In [17]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'

In [19]:
%%time
for i in range(len(note_nlp)):
    
    table_id = note_nlp[i]
    prefix_date = date_column_prefix[i]
    concept_id_prefix_table = concept_id_prefix[i]
    
    print(f'Subsetting {table_id}')
    
    sql = """
    CREATE OR REPLACE TABLE
     `som-nero-nigam-starr.cdm_subset.note_nlp` AS
    SELECT
    c.concept_name as _note_nlp_name,
    table_.*
    FROM
     `som-nero-nigam-starr.starr_omop_cdm5_deid_20211213.note_nlp` table_
    JOIN
     `som-nero-nigam-starr.starr_omop_cdm5_deid_20211213.concept` c
    ON
     c.concept_id = table_.note_nlp_concept_id
    JOIN
         `som-nero-nigam-starr.cdm_subset.note` note
    ON note.note_id = table_.note_id
"""
    client.query(sql).result();

Subsetting note_nlp
CPU times: user 35.5 ms, sys: 5.8 ms, total: 41.3 ms
Wall time: 3min 35s


Tables that we will copy by filtering only on person_id

In [ ]:
tables_filter_by_person_id = ["condition_era",
                              "death",
                              "dose_era",
                              "drug_era",
                              "observation_period",
                              "person"]

concept_id_prefix = ["condition",
                     "death_type",
                     "drug",
                     "drug",
                     "period_type",
                     "gender"]

In [ ]:
%%time
for i in range(len(tables_filter_by_person_id)):
    
    table_id = tables_filter_by_person_id[i]
    concept_id_prefix_table = concept_id_prefix[i]
    
    print(f'Subsetting {table_id}')
    if table_id != 'location':
        sql = """
        CREATE OR REPLACE TABLE
        `{work_project_id}.{cdm_subset_dataset_id}.{table_id}` AS
        SELECT
        c.concept_name as _{concept_id_prefix}_name,
        table_.*
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` table_
        JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` c
        ON
        c.concept_id = table_.{concept_id_prefix}_concept_id
        JOIN
        `{work_project_id}.kbechler_explore.{cohort_table_id}` cohort
        ON
        table_.person_id = cohort.subject_id
        WHERE
        cohort_definition_id = {cohort_id}
        """.format_map({'work_project_id': work_project_id,
                        'work_dataset_id': work_dataset_id,
                        'cdm_subset_dataset_id': cdm_subset_dataset_id,
                        'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'cohort_table_id': cohort_table_id,
                        'cohort_id': cohort_id,
                        'table_id': table_id,
                        'concept_id_prefix': concept_id_prefix_table})

        client.query(sql).result();

Subset the location table

In [ ]:
sql=""" CREATE OR REPLACE TABLE `{work_project_id}.{cdm_subset_dataset_id}.location` AS
        WITH t1 AS
        (  SELECT 
            DISTINCT
            location_id
           FROM 
            `{work_project_id}.{cdm_subset_dataset_id}.person`
        ),
        t2 AS
        (
            SELECT
             DISTINCT 
              location_id
            FROM
             `{work_project_id}.{cdm_subset_dataset_id}.care_site`
        ),
        t3 AS
        (
            SELECT
             location_id
            FROM
             t1
            UNION ALL
            SELECT
             location_id
            FROM t2
        )
        SELECT 
         loc.*
        FROM 
         `{cdm_project_id}.{cdm_dataset_id}.location` loc
        WHERE
         location_id IN (SELECT DISTINCT location_id FROM t3)
    """.format_map({'work_project_id': work_project_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id})


client.query(sql).result()

In [20]:
tables_to_download = [
                      #"care_site",
                      #"cdm_source",
                      #"concept", # Big Vocab table
                      #"concept_ancestor", # Big Vocabulary Table
                      #"concept_class", # Big Vocabulary Table
                      #"concept_relationship", # Big Vocabulary Table
                      #"concept_synonym", # Vocab Table
                      #"condition_era",
                      #"condition_occurrence",
                      #"cost", # empty 
                      #"death",
                      #"device_exposure",
                      #"domain", # Vocab Table
                      #"dose_era",
                      #"drug_era",
                      #"drug_exposure",
                      #"drug_strength", # Vocab Table
                      #"fact_relationship",                  
                      #"location",
                      #"measurement",
                      #"metadata",
                      #"note"] 
                      "note_nlp"] # Big Table in general
                      #"observation",
                      #"observation_period",
                      #"payer_plan_period", # empty
                      #"person",
                      #"procedure_occurrence"]
                      #"provider",
                      #"relationship", # Vocab Table
                      #"source_to_concept_map", # Vocab Table
                      #"specimen", # empty
                      #"visit_detail",
                      #"visit_occurrence",
                      #"vocabulary"] # Vocab Table

In [21]:
for table_id in tables_to_download:
    
    sql = """
    SELECT 
     COUNT(*) as counts
    FROM
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}`
    """.format_map({'work_project_id': work_project_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'table_id': table_id})
    
    df = client.query(sql).to_dataframe()
    
    print(f"{df.values[0][0]} rows for {table_id}")

/home/kbechler/.conda/envs/env_name/lib/python3.9/site-packages/google/cloud/bigquery/client.py:461: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


62605193 rows for note_nlp


In [22]:
download_folder = "/home/kbechler/notebooks/data"

In [23]:
%%time
for i in range(len(tables_to_download)):
    table_id = tables_to_download[i]
    
    # get the column names to ensure pandas do not do any conversion
    table_ref = client.get_table(f'{work_project_id}.{cdm_subset_dataset_id}.{table_id}')
    schema = {schema.name:'object' for schema in table_ref.schema}
    
    file_name = f"{download_folder}/{table_id}.tsv"
    print(f'Downloading {work_project_id}.{cdm_subset_dataset_id}.{table_id}')
    print(f" to file {file_name}")
    
    sql = """
    SELECT
     *
    FROM
     `{work_project_id}.{cdm_subset_dataset_id}.{table_id}`
    """.format_map({'work_project_id': work_project_id,
                    'cdm_subset_dataset_id': cdm_subset_dataset_id,
                    'table_id': table_id})
    
    client.query(sql).to_dataframe(dtypes=schema,
                                   progress_bar_type='tqdm').to_csv(path_or_buf=file_name,
                                                                    index=False,
                                                                    sep='\t')

 to file /home/kbechler/notebooks/data/note_nlp.tsv


Query complete after 42.51s: : 6query [00:42,  7.08s/query]                                                          
Downloading: 100%|██████████| 62605193/62605193 [5:02:46<00:00, 3446.28rows/s]  


CPU times: user 59min 57s, sys: 1min 36s, total: 1h 1min 34s
Wall time: 5h 39min 24s
